# EMBEDDING

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Load multilingual SBERT model (MPNet backbone, supports Slovene)
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model.eval()

# Load your dataset
df = pd.read_json("rtvslo_validation.json")  # assumes a list of article dicts

df['full_text'] = df['title'] + " " + df['lead'] + " " + df['paragraphs'].apply(lambda x: " ".join(x))

# Encode all articles
embeddings = []
for text in tqdm(df['full_text']):
    emb = model.encode(text, convert_to_tensor=True)  # returns a torch.Tensor
    embeddings.append(emb)

# Stack into one tensor
embeddings_tensor = torch.stack(embeddings)
torch.save(embeddings_tensor, "text_embeddings_val.pt")

# Also save the targets
targets = torch.tensor(df['n_comments'].values, dtype=torch.float32)
torch.save(targets, "targets_val.pt")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Slovene SloBERTa model
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/sloberta")
model = AutoModel.from_pretrained("EMBEDDIA/sloberta").to(device)
model.eval()

# Load dataset
df = pd.read_json("rtvslo_test.json")
df['full_text'] = df['title'] + " " + df['lead'] + " " + df['paragraphs'].apply(lambda x: " ".join(x))

# Optional: Mean pooling function
def mean_pooling(outputs, attention_mask):
    token_embeddings = outputs.last_hidden_state  # (batch, seq_len, hidden)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, dim=1) / \
           torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)

# Function to embed a single article using mean pooling
def embed_article(text):
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=512,
        return_token_type_ids=False
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        pooled = mean_pooling(outputs, inputs['attention_mask'])
    return pooled.squeeze(0)

# Embed all articles
embeddings = []
for text in tqdm(df['full_text']):
    emb = embed_article(text)
    embeddings.append(emb.cpu())  # move back to CPU to stack later

# Stack and save embeddings
embeddings_tensor = torch.stack(embeddings)
torch.save(embeddings_tensor, "sloberta_embeddings_final.pt")


Some weights of CamembertModel were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2218/2218 [01:16<00:00, 29.01it/s]


# BAGGING

## + calibration

In [ ]:
# --- 1. Imports ---
import json
import numpy as np
import pandas as pd
import torch
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error

# --- 2. Utility Functions ---
def load_json(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_topics_subtopics(articles):
    topics, subtopics = [], []
    for a in articles:
        parts = a['url'].split('/')
        topic = parts[3] if len(parts) > 3 else 'none'
        subtopic = parts[4] if len(parts) > 4 else 'none'
        topics.append(topic)
        subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")
    return topics, subtopics

def enrich_articles(articles):
    for a in articles:
        dt = pd.to_datetime(a['date'])
        a['year'] = dt.year
        a['month'] = dt.month
        a['day_of_week'] = dt.weekday()
        a['hour'] = dt.hour
    return articles

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)
    return date_feats

def prepare_features(articles, bert_reduced, topic_enc=None, subtopic_enc=None, fit_encoders=False):
    topics, subtopics = extract_topics_subtopics(articles)

    if fit_encoders:
        topic_enc = LabelEncoder().fit(topics)
        subtopic_enc = LabelEncoder().fit(subtopics)

    topics = [t if t in topic_enc.classes_ else topic_enc.classes_[0] for t in topics]
    subtopics = [s if s in subtopic_enc.classes_ else subtopic_enc.classes_[0] for s in subtopics]

    topic_ids = topic_enc.transform(topics)
    subtopic_ids = subtopic_enc.transform(subtopics)

    time_feats = process_date_features(articles)

    features = np.hstack([bert_reduced, topic_ids[:, None], subtopic_ids[:, None], time_feats])
    return features, topic_enc, subtopic_enc

def train_lightgbm(X_train, y_train, X_val, y_val, weights_train, weights_val):
    params = {
        'objective': 'quantile',     # quantile regression (median)
        'alpha': 0.5,                # 0.5 = median
        'learning_rate': 0.05,
        'num_leaves': 128,
        'max_depth': -1,
        'min_data_in_leaf': 30,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'device_type': 'cpu',
        'verbosity': -1,
        'seed': 42
    }

    train_set = lgb.Dataset(X_train, label=y_train, weight=weights_train)
    val_set = lgb.Dataset(X_val, label=y_val, weight=weights_val)

    model = lgb.train(
        params,
        train_set,
        valid_sets=[train_set, val_set],
        valid_names=['train', 'valid'],
        num_boost_round=5000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=100)
        ]
    )

    return model

def calibrate_predictions(preds, train_targets_sorted, pred_train_sorted):
    """Postprocessing: Match percentiles between train and predictions."""
    ranks = np.searchsorted(pred_train_sorted, preds, side='left') / len(pred_train_sorted)
    calibrated = np.quantile(train_targets_sorted, ranks)
    return calibrated

# --- 3. Load and Prepare Data ---
print("🔵 Loading data...")
train_articles = load_json("rtvslo_train.json")
val_articles = load_json("rtvslo_validation.json")

train_articles = enrich_articles(train_articles)
val_articles = enrich_articles(val_articles)

bert_train = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()
bert_val = torch.load("sloberta_embeddings_val.pt", map_location="cpu").numpy()

# --- 4. Dimensionality Reduction ---
print("🔵 Running PCA...")
pca_model = PCA(n_components=50, random_state=42)
bert_train_reduced = pca_model.fit_transform(bert_train)
bert_val_reduced = pca_model.transform(bert_val)

print(f"✅ PCA done. Shape: {bert_train_reduced.shape}")

# --- 5. Prepare Features
print("🔵 Preparing features...")
X_train, topic_enc, subtopic_enc = prepare_features(train_articles, bert_train_reduced, fit_encoders=True)
X_val, _, _ = prepare_features(val_articles, bert_val_reduced, topic_enc, subtopic_enc)

y_train_raw = np.array([a['n_comments'] for a in train_articles], dtype=np.float32)
np.savetxt("train_targets.txt", y_train_raw, fmt="%.4f")

y_val_raw = np.array([a['n_comments'] for a in val_articles], dtype=np.float32)

y_train = np.log1p(y_train_raw)
y_val = np.log1p(y_val_raw)

# Weights: higher for larger original n_comments
weights_train = np.sqrt(y_train_raw + 1)
weights_val = np.sqrt(y_val_raw + 1)

# --- 6. Train LightGBM
print("🔵 Training LightGBM...")
model = train_lightgbm(X_train, y_train, X_val, y_val, weights_train, weights_val)

# --- 7. Predict
print("🔵 Predicting...")
val_preds = model.predict(X_val, num_iteration=model.best_iteration)

# Undo log1p
val_preds_exp = np.clip(np.expm1(val_preds), 0, None)

# --- 8. Postprocessing Calibration
print("🔵 Calibrating predictions...")
pred_train = model.predict(X_train, num_iteration=model.best_iteration)
train_preds_exp = np.clip(np.expm1(pred_train), 0, None)

train_targets_sorted = np.sort(y_train_raw)
pred_train_sorted = np.sort(train_preds_exp)

val_preds_calibrated = calibrate_predictions(val_preds_exp, train_targets_sorted, pred_train_sorted)

# --- 9. Save and Evaluate
print("🔵 Saving predictions...")
np.savetxt("predictions_val_raw.txt", val_preds_exp, fmt="%.4f")
np.savetxt("predictions_val_calibrated.txt", val_preds_calibrated, fmt="%.4f")

print("🔵 Evaluating...")
mae_raw = mean_absolute_error(y_val_raw, val_preds_exp)
mae_calibrated = mean_absolute_error(y_val_raw, val_preds_calibrated)

print(f"✅ MAE Raw          : {mae_raw:.4f}")
print(f"✅ MAE Calibrated   : {mae_calibrated:.4f}")


🔵 Loading data...
🔵 Running PCA...
✅ PCA done. Shape: (46830, 50)
🔵 Preparing features...
🔵 Training LightGBM...
Training until validation scores don't improve for 100 rounds
[100]	train's quantile: 0.312468	valid's quantile: 0.391089
[200]	train's quantile: 0.268099	valid's quantile: 0.37803
[300]	train's quantile: 0.248439	valid's quantile: 0.375506
[400]	train's quantile: 0.237404	valid's quantile: 0.373446
[500]	train's quantile: 0.229766	valid's quantile: 0.372943
[600]	train's quantile: 0.222879	valid's quantile: 0.372449
[700]	train's quantile: 0.217853	valid's quantile: 0.371897
[800]	train's quantile: 0.212485	valid's quantile: 0.372245
Early stopping, best iteration is:
[767]	train's quantile: 0.214405	valid's quantile: 0.371737
🔵 Predicting...
🔵 Calibrating predictions...
🔵 Saving predictions...
🔵 Evaluating...
✅ MAE Raw          : 29.9895
✅ MAE Calibrated   : 28.9768


In [ ]:
# --- 1. Imports ---
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)
    return date_feats

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, bert_vectors, topic_ids, subtopic_ids, date_feats, targets):
        self.X = bert_vectors
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.y[idx]

# --- 4. Model ---
class MLPWithEmbeddings(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats], dim=1)
        return self.model(x)

# --- 5. Main Trainer Class ---
class RTVSloBERT:
    def __init__(self, batch_size=170, epochs=70, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05, model_id=0):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_id = model_id

    def fit(self, train_data):
        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = [a['n_comments'] for a in train_data]
        targets = [np.log1p(t) for t in raw_targets]
        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics = []
        subtopics = []
        for a in train_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )
        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithEmbeddings(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_)).to(self.device)

        self._train(self.model, train_loader, val_loader)

    def _train(self, model, train_loader, val_loader):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.HuberLoss(delta=5.0)

        best_val_mae = float('inf')
        best_model_state = None

        for epoch in range(self.epochs):
            model.train()
            for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

            model.eval()
            val_losses = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in val_loader:
                    y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                    loss = criterion(y_pred, y_batch)
                    val_losses.append(loss.item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            if val_loss < best_val_mae:
                best_val_mae = val_loss
                best_model_state = model.state_dict()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{self.epochs} - Validation loss: {val_loss:.4f}")

        # Save snapshot
        os.makedirs("snapshots", exist_ok=True)
        torch.save(best_model_state, f"snapshots/model_{self.model_id:03d}.pt")

    def predict(self, test_data, bert_vectors):
        for a in test_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        topics = []
        subtopics = []
        for a in test_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(test_data)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = self.topic_encoder.transform(topic_ids)
        subtopic_ids = self.subtopic_encoder.transform(subtopic_ids)

        X = torch.tensor(bert_vectors, dtype=torch.float32).to(self.device)
        topic_ids = torch.tensor(topic_ids, dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long).to(self.device)
        date_feats = torch.tensor(date_feats, dtype=torch.float32).to(self.device)

        self.model.eval()
        with torch.no_grad():
            preds = self.model(X, topic_ids, subtopic_ids, date_feats).squeeze().cpu().numpy()
            return np.clip(np.expm1(preds), 0, None)

# --- 6. Main Loop (Ensemble 70×) ---
if __name__ == '__main__':
    train = load("rtvslo_train.json")
    validation = load("rtvslo_validation.json")
    test = load("rtvslo_test.json")

    bert_vectors_validation = torch.load("sloberta_embeddings_val.pt", map_location="cpu").numpy()
    bert_vectors_test = torch.load("sloberta_embeddings_final.pt", map_location="cpu").numpy()

    preds_validation = []
    preds_test = []

    for seed in range(20):
        print(f"\n🚀 Training model {seed+1}/70...")
        model = RTVSloBERT(model_id=seed)
        model.fit(train)

        val_preds = model.predict(validation, bert_vectors_validation)
        test_preds = model.predict(test, bert_vectors_test)

        preds_validation.append(val_preds)
        preds_test.append(test_preds)

    # Average all predictions
    final_val_preds = np.median(preds_validation, axis=0)
    final_test_preds = np.median(preds_test, axis=0)

    np.savetxt("final_predictions_val.txt", final_val_preds, fmt="%.4f")
    np.savetxt("final_predictions_test.txt", final_test_preds, fmt="%.4f")

    print("✅ Done!")



🚀 Training model 1/70...
Epoch 10/70 - Validation loss: 0.4572
Epoch 20/70 - Validation loss: 0.4427
Epoch 30/70 - Validation loss: 0.4370
Epoch 40/70 - Validation loss: 0.4420
Epoch 50/70 - Validation loss: 0.4338
Epoch 60/70 - Validation loss: 0.4317
Epoch 70/70 - Validation loss: 0.4329

🚀 Training model 2/70...
Epoch 10/70 - Validation loss: 0.4610
Epoch 20/70 - Validation loss: 0.4479
Epoch 30/70 - Validation loss: 0.4452
Epoch 40/70 - Validation loss: 0.4339
Epoch 50/70 - Validation loss: 0.4396
Epoch 60/70 - Validation loss: 0.4395
Epoch 70/70 - Validation loss: 0.4395

🚀 Training model 3/70...
Epoch 10/70 - Validation loss: 0.4614
Epoch 20/70 - Validation loss: 0.4607
Epoch 30/70 - Validation loss: 0.4514
Epoch 40/70 - Validation loss: 0.4396
Epoch 50/70 - Validation loss: 0.4495
Epoch 60/70 - Validation loss: 0.4346
Epoch 70/70 - Validation loss: 0.4347

🚀 Training model 4/70...
Epoch 10/70 - Validation loss: 0.4661
Epoch 20/70 - Validation loss: 0.4540
Epoch 30/70 - Validati

# SEMANTIC FLAGS

In [ ]:
# --- 1. Imports ---
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os
import pandas as pd
from tqdm import tqdm

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate(
        [years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]],
        axis=1
    )
    return date_feats

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

def extract_semantic_flags(articles):
    flags = []
    for a in articles:
        url = a.get('url', '').lower()
        is_politics = any(word in url for word in ['politics', 'elections', 'parliament', 'vlada', 'politika'])
        is_ukraine_russia = any(word in url for word in ['ukraine', 'russia', 'ukrajina', 'rusija'])
        is_israel_palestine = any(word in url for word in ['israel', 'palestine', 'gaza', 'hamas'])
        is_sport = any(word in url for word in ['sport', 'nogomet', 'košarka', 'tenis', 'hokej'])
        flags.append([is_politics, is_ukraine_russia, is_israel_palestine, is_sport])
    return np.array(flags, dtype=np.float32)

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, X, topic_ids, subtopic_ids, date_feats, semantic_flags, targets):
        self.X = X
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.semantic_flags = semantic_flags
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.semantic_flags[idx], self.y[idx]

# --- 4. Model ---
class MLPWithEmbeddings(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6 + 4, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats, semantic_flags):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats, semantic_flags], dim=1)
        return self.model(x)

# --- 5. Main Trainer ---
class RTVSloBERT:
    def __init__(self, batch_size=170, epochs=70, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def fit(self, train_data, save_dir="snapshots"):
        os.makedirs(save_dir, exist_ok=True)

        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = np.array([a['n_comments'] for a in train_data])
        targets = np.log1p(raw_targets)

        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics = []
        subtopics = []
        for a in train_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(train_data)
        semantic_flags = extract_semantic_flags(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, semantic_train, semantic_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_flags, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )

        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithEmbeddings(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_)).to(self.device)

        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.HuberLoss(delta=5.0)

        for epoch in range(self.epochs):
            self.model.train()
            for x_batch, topic_ids, subtopic_ids, date_feats, semantic_flags, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = self.model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_flags).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

            self.model.eval()
            val_losses = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, semantic_flags, y_batch in val_loader:
                    y_pred = self.model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_flags).squeeze()
                    loss = criterion(y_pred, y_batch)
                    val_losses.append(loss.item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            print(f"Epoch {epoch+1} - Val Loss: {val_loss:.4f}")

            # Save snapshot
            torch.save(self.model.state_dict(), os.path.join(save_dir, f"snapshot_epoch{epoch+1}.pt"))

    def predict(self, articles, bert_vectors_path):
        bert_vectors = torch.load(bert_vectors_path, map_location="cpu").to(self.device)

        for a in articles:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        topics = []
        subtopics = []
        for a in articles:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = torch.tensor(process_date_features(articles), dtype=torch.float32).to(self.device)
        semantic_flags = torch.tensor(extract_semantic_flags(articles), dtype=torch.float32).to(self.device)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = torch.tensor(self.topic_encoder.transform(topic_ids), dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(self.subtopic_encoder.transform(subtopic_ids), dtype=torch.long).to(self.device)

        self.model.eval()
        preds = []
        with torch.no_grad():
            for i in tqdm(range(0, bert_vectors.shape[0], self.batch_size)):
                xb = bert_vectors[i:i+self.batch_size]
                tb = topic_ids[i:i+self.batch_size]
                stb = subtopic_ids[i:i+self.batch_size]
                db = date_feats[i:i+self.batch_size]
                sb = semantic_flags[i:i+self.batch_size]
                yb = self.model(xb, tb, stb, db, sb).squeeze()
                preds.append(yb)

        preds = torch.cat(preds).cpu().numpy()
        preds = np.expm1(np.clip(preds, 0, None))
        return preds

# --- 6. Main ---
if __name__ == "__main__":
    train_data = load("rtvslo_train.json")
    val_data = load("rtvslo_validation.json")

    model = RTVSloBERT()
    model.fit(train_data)

    preds_val = model.predict(val_data, "sloberta_embeddings_val.pt")
    np.savetxt("predictions_val_semantic_noweights.txt", preds_val, fmt="%.4f")


Epoch 1 - Val Loss: 0.5442
Epoch 2 - Val Loss: 0.4991
Epoch 3 - Val Loss: 0.5044
Epoch 4 - Val Loss: 0.4735
Epoch 5 - Val Loss: 0.4660
Epoch 6 - Val Loss: 0.4719
Epoch 7 - Val Loss: 0.4594
Epoch 8 - Val Loss: 0.4543
Epoch 9 - Val Loss: 0.4592
Epoch 10 - Val Loss: 0.4543
Epoch 11 - Val Loss: 0.4579
Epoch 12 - Val Loss: 0.4537
Epoch 13 - Val Loss: 0.4685
Epoch 14 - Val Loss: 0.4496
Epoch 15 - Val Loss: 0.4418
Epoch 16 - Val Loss: 0.4492
Epoch 17 - Val Loss: 0.4438
Epoch 18 - Val Loss: 0.4541
Epoch 19 - Val Loss: 0.4439
Epoch 20 - Val Loss: 0.4465
Epoch 21 - Val Loss: 0.4634
Epoch 22 - Val Loss: 0.4441
Epoch 23 - Val Loss: 0.4455
Epoch 24 - Val Loss: 0.4410
Epoch 25 - Val Loss: 0.4375
Epoch 26 - Val Loss: 0.4373
Epoch 27 - Val Loss: 0.4407
Epoch 28 - Val Loss: 0.4363
Epoch 29 - Val Loss: 0.4357
Epoch 30 - Val Loss: 0.4379
Epoch 31 - Val Loss: 0.4455
Epoch 32 - Val Loss: 0.4461
Epoch 33 - Val Loss: 0.4317
Epoch 34 - Val Loss: 0.4419
Epoch 35 - Val Loss: 0.4372
Epoch 36 - Val Loss: 0.4441
E

100%|██████████| 14/14 [00:00<00:00, 1675.71it/s]


# CONTEXT

In [ ]:
# --- 1. Imports ---
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
import os

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def enrich_articles(articles):
    """Extract date features + semantic groups + article context features."""
    # Step 1: Extract datetime features
    for a in articles:
        dt = pd.to_datetime(a['date'])
        a['year'] = dt.year
        a['month'] = dt.month
        a['day_of_week'] = dt.weekday()
        a['hour'] = dt.hour
        a['timestamp'] = dt.timestamp()

    # Step 2: Semantic group flags
    for a in articles:
        url = a.get('url', '').lower()
        a['is_politics'] = int(any(word in url for word in ['politics', 'elections', 'parliament', 'vlada', 'politika']))
        a['is_ukraine_russia'] = int(any(word in url for word in ['ukraine', 'russia', 'ukrajina', 'rusija']))
        a['is_israel_palestine'] = int(any(word in url for word in ['israel', 'palestine', 'gaza', 'hamas']))
        a['is_sport'] = int(any(word in url for word in ['sport', 'nogomet', 'košarka', 'tenis', 'hokej']))

    # Step 3: Sort articles
    articles.sort(key=lambda a: a['timestamp'])

    # Step 4: Context window features
    timestamps = np.array([a['timestamp'] for a in articles])
    n_comments = np.array([a['n_comments'] for a in articles])

    for i, a in enumerate(articles):
        # Last 1h, 6h, 12h windows
        t = a['timestamp']
        a['articles_last_1h'] = np.sum((t - timestamps[:i]) <= 3600)
        a['articles_last_6h'] = np.sum((t - timestamps[:i]) <= 6*3600)
        a['articles_last_12h'] = np.sum((t - timestamps[:i]) <= 12*3600)
        # Total comments in last 6h
        mask_6h = (t - timestamps[:i]) <= 6*3600
        a['total_comments_last_6h'] = np.sum(n_comments[:i][mask_6h]) if np.any(mask_6h) else 0.0

    return articles

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)
    return date_feats

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, X, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats, targets):
        self.X = X
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.semantic_flags = semantic_flags
        self.context_feats = context_feats
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.semantic_flags[idx], self.context_feats[idx], self.y[idx]

# --- 4. Model ---
class MLPWithContext(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6 + 4 + 4, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats, semantic_flags, context_feats], dim=1)
        return self.model(x)

# --- 5. Main Trainer ---
class RTVSloBERTContext:
    def __init__(self, batch_size=170, epochs=150, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def fit(self, train_data):
        os.makedirs("snapshots", exist_ok=True)
        train_data = enrich_articles(train_data)

        raw_targets = np.array([a['n_comments'] for a in train_data])
        targets = np.log1p(raw_targets)

        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in train_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(train_data)
        semantic_flags = np.array([[a['is_politics'], a['is_ukraine_russia'], a['is_israel_palestine'], a['is_sport']] for a in train_data], dtype=np.float32)
        context_feats = np.array([[a['articles_last_1h'], a['articles_last_6h'], a['articles_last_12h'], a['total_comments_last_6h']] for a in train_data], dtype=np.float32)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, semantic_train, semantic_val, context_train, context_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_train, dtype=torch.float32).to(self.device),
            torch.tensor(context_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )

        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_val, dtype=torch.float32).to(self.device),
            torch.tensor(context_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithContext(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_)).to(self.device)

        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        criterion = nn.HuberLoss(delta=5.0)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

        best_loss = float('inf')
        patience_counter = 0
        patience_limit = 15

        for epoch in range(self.epochs):
            self.model.train()
            for xb, topicb, subtopicb, dateb, semanticb, contextb, yb in train_loader:
                optimizer.zero_grad()
                preds = self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze()
                loss = criterion(preds, yb)
                loss.backward()
                optimizer.step()

            # Validation
            self.model.eval()
            val_losses = []
            with torch.no_grad():
                for xb, topicb, subtopicb, dateb, semanticb, contextb, yb in val_loader:
                    preds = self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze()
                    val_losses.append(criterion(preds, yb).item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            print(f"Epoch {epoch+1} - Validation Loss: {val_loss:.4f}")

            if val_loss < best_loss - 1e-4:
                best_loss = val_loss
                patience_counter = 0
                torch.save(self.model.state_dict(), f"snapshots/best_model.pt")
            else:
                patience_counter += 1
                if patience_counter >= patience_limit:
                    print("Early stopping triggered.")
                    break

    def predict(self, articles, bert_path):
        articles = enrich_articles(articles)
        bert_vectors = torch.load(bert_path, map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in articles])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(articles)
        semantic_flags = np.array([[a['is_politics'], a['is_ukraine_russia'], a['is_israel_palestine'], a['is_sport']] for a in articles], dtype=np.float32)
        context_feats = np.array([[a['articles_last_1h'], a['articles_last_6h'], a['articles_last_12h'], a['total_comments_last_6h']] for a in articles], dtype=np.float32)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = self.topic_encoder.transform(topic_ids)
        subtopic_ids = self.subtopic_encoder.transform(subtopic_ids)

        X = torch.tensor(bert_vectors, dtype=torch.float32).to(self.device)
        topic_ids = torch.tensor(topic_ids, dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long).to(self.device)
        date_feats = torch.tensor(date_feats, dtype=torch.float32).to(self.device)
        semantic_flags = torch.tensor(semantic_flags, dtype=torch.float32).to(self.device)
        context_feats = torch.tensor(context_feats, dtype=torch.float32).to(self.device)

        self.model.load_state_dict(torch.load("snapshots/best_model.pt"))
        self.model.eval()

        preds = []
        with torch.no_grad():
            for i in tqdm(range(0, len(X), self.batch_size)):
                xb = X[i:i+self.batch_size]
                topicb = topic_ids[i:i+self.batch_size]
                subtopicb = subtopic_ids[i:i+self.batch_size]
                dateb = date_feats[i:i+self.batch_size]
                semanticb = semantic_flags[i:i+self.batch_size]
                contextb = context_feats[i:i+self.batch_size]
                preds.append(self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze())

        preds = torch.cat(preds).cpu().numpy()
        preds = np.expm1(np.clip(preds, 0, None))
        return preds

# --- 6. Main ---
if __name__ == "__main__":
    train_articles = load("rtvslo_train.json")
    val_articles = load("rtvslo_validation.json")

    model = RTVSloBERTContext()
    model.fit(train_articles)

    preds_val = model.predict(val_articles, "sloberta_embeddings_val.pt")
    np.savetxt("predictions_val_context.txt", preds_val, fmt="%.4f")


Epoch 1 - Validation Loss: 0.7560
Epoch 2 - Validation Loss: 0.6287
Epoch 3 - Validation Loss: 0.6443
Epoch 4 - Validation Loss: 0.5695
Epoch 5 - Validation Loss: 0.5657
Epoch 6 - Validation Loss: 0.5712
Epoch 7 - Validation Loss: 0.5596
Epoch 8 - Validation Loss: 0.5676
Epoch 9 - Validation Loss: 0.5372
Epoch 10 - Validation Loss: 0.5434
Epoch 11 - Validation Loss: 0.5410
Epoch 12 - Validation Loss: 0.6468
Epoch 13 - Validation Loss: 0.5878
Epoch 14 - Validation Loss: 0.5256
Epoch 15 - Validation Loss: 0.5226
Epoch 16 - Validation Loss: 0.5778
Epoch 17 - Validation Loss: 0.5413
Epoch 18 - Validation Loss: 0.5528
Epoch 19 - Validation Loss: 0.8377
Epoch 20 - Validation Loss: 0.5065
Epoch 21 - Validation Loss: 0.6112
Epoch 22 - Validation Loss: 0.5119
Epoch 23 - Validation Loss: 0.5352
Epoch 24 - Validation Loss: 0.5259
Epoch 25 - Validation Loss: 0.5080
Epoch 26 - Validation Loss: 0.4983
Epoch 27 - Validation Loss: 0.5150
Epoch 28 - Validation Loss: 0.4926
Epoch 29 - Validation Loss: 0

100%|██████████| 14/14 [00:00<00:00, 1564.33it/s]


## ADDITIONAL MLP

In [ ]:
# --- 1. Imports ---
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
import os

# --- 2. Utility functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def enrich_articles(articles):
    for a in articles:
        dt = pd.to_datetime(a['date'])
        a['year'] = dt.year
        a['month'] = dt.month
        a['day_of_week'] = dt.weekday()
        a['hour'] = dt.hour
        a['timestamp'] = dt.timestamp()

        url = a.get('url', '').lower()
        a['is_politics'] = int(any(word in url for word in ['politics', 'elections', 'parliament', 'vlada', 'politika']))
        a['is_ukraine_russia'] = int(any(word in url for word in ['ukraine', 'russia', 'ukrajina', 'rusija']))
        a['is_israel_palestine'] = int(any(word in url for word in ['israel', 'palestine', 'gaza', 'hamas']))
        a['is_sport'] = int(any(word in url for word in ['sport', 'nogomet', 'košarka', 'tenis', 'hokej']))

    articles.sort(key=lambda a: a['timestamp'])

    timestamps = np.array([a['timestamp'] for a in articles])
    n_comments = np.array([a['n_comments'] for a in articles])

    for i, a in enumerate(articles):
        t = a['timestamp']
        a['articles_last_1h'] = np.sum((t - timestamps[:i]) <= 3600)
        a['articles_last_6h'] = np.sum((t - timestamps[:i]) <= 6*3600)
        a['articles_last_12h'] = np.sum((t - timestamps[:i]) <= 12*3600)
        mask_6h = (t - timestamps[:i]) <= 6*3600
        a['total_comments_last_6h'] = np.sum(n_comments[:i][mask_6h]) if np.any(mask_6h) else 0.0

    return articles

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    return np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, X, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats, targets):
        self.X = X
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.semantic_flags = semantic_flags
        self.context_feats = context_feats
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.semantic_flags[idx], self.context_feats[idx], self.y[idx]

# --- 4. Model ---
class MLPBlock(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.norm = nn.LayerNorm(input_dim)

    def forward(self, x):
        x = self.norm(x)
        return self.mlp(x)

class MLPWithContextFeatures(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)
        self.context_mlp = MLPBlock(8)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6 + 8, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        context_input = torch.cat([semantic_flags, context_feats], dim=-1)
        context_out = self.context_mlp(context_input)

        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats, context_out], dim=1)
        return self.model(x)

# --- 5. Trainer ---
class RTVSloImprovedModel:
    def __init__(self, batch_size=170, epochs=150, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def fit(self, train_data):
        os.makedirs("snapshots", exist_ok=True)
        train_data = enrich_articles(train_data)

        raw_targets = np.array([a['n_comments'] for a in train_data])
        targets = np.log1p(raw_targets)

        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in train_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(train_data)
        semantic_flags = np.array([[a['is_politics'], a['is_ukraine_russia'], a['is_israel_palestine'], a['is_sport']] for a in train_data], dtype=np.float32)
        context_feats = np.array([[a['articles_last_1h'], a['articles_last_6h'], a['articles_last_12h'], a['total_comments_last_6h']] for a in train_data], dtype=np.float32)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, semantic_train, semantic_val, context_train, context_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_flags, context_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_train, dtype=torch.float32).to(self.device),
            torch.tensor(context_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )

        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_val, dtype=torch.float32).to(self.device),
            torch.tensor(context_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithContextFeatures(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_)).to(self.device)

        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        criterion = nn.HuberLoss(delta=5.0)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

        best_loss = float('inf')
        patience_counter = 0
        patience_limit = 15

        for epoch in range(self.epochs):
            self.model.train()
            for xb, topicb, subtopicb, dateb, semanticb, contextb, yb in train_loader:
                optimizer.zero_grad()
                preds = self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze()
                loss = criterion(preds, yb)
                loss.backward()
                optimizer.step()

            self.model.eval()
            val_losses = []
            with torch.no_grad():
                for xb, topicb, subtopicb, dateb, semanticb, contextb, yb in val_loader:
                    preds = self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze()
                    val_losses.append(criterion(preds, yb).item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            print(f"Epoch {epoch+1} - Validation Loss: {val_loss:.4f}")

            if val_loss < best_loss - 1e-4:
                best_loss = val_loss
                patience_counter = 0
                torch.save(self.model.state_dict(), "snapshots/best_improved_model.pt")
            else:
                patience_counter += 1
                if patience_counter >= patience_limit:
                    print("Early stopping triggered.")
                    break

    def predict(self, articles, bert_path):
        articles = enrich_articles(articles)
        bert_vectors = torch.load(bert_path, map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in articles])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(articles)
        semantic_flags = np.array([[a['is_politics'], a['is_ukraine_russia'], a['is_israel_palestine'], a['is_sport']] for a in articles], dtype=np.float32)
        context_feats = np.array([[a['articles_last_1h'], a['articles_last_6h'], a['articles_last_12h'], a['total_comments_last_6h']] for a in articles], dtype=np.float32)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = torch.tensor(self.topic_encoder.transform(topic_ids), dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(self.subtopic_encoder.transform(subtopic_ids), dtype=torch.long).to(self.device)

        X = torch.tensor(bert_vectors, dtype=torch.float32).to(self.device)
        date_feats = torch.tensor(date_feats, dtype=torch.float32).to(self.device)
        semantic_flags = torch.tensor(semantic_flags, dtype=torch.float32).to(self.device)
        context_feats = torch.tensor(context_feats, dtype=torch.float32).to(self.device)

        self.model.load_state_dict(torch.load("snapshots/best_improved_model.pt"))
        self.model.eval()

        preds = []
        with torch.no_grad():
            for i in tqdm(range(0, len(X), self.batch_size)):
                xb = X[i:i+self.batch_size]
                topicb = topic_ids[i:i+self.batch_size]
                subtopicb = subtopic_ids[i:i+self.batch_size]
                dateb = date_feats[i:i+self.batch_size]
                semanticb = semantic_flags[i:i+self.batch_size]
                contextb = context_feats[i:i+self.batch_size]
                preds.append(self.model(xb, topicb, subtopicb, dateb, semanticb, contextb).squeeze())

        preds = torch.cat(preds).cpu().numpy()
        preds = np.expm1(np.clip(preds, 0, None))
        return preds

# --- 6. Main ---
if __name__ == "__main__":
    train_articles = load("rtvslo_train.json")
    val_articles = load("rtvslo_validation.json")

    model = RTVSloImprovedModel()
    model.fit(train_articles)

    preds_val = model.predict(val_articles, "sloberta_embeddings_val.pt")
    np.savetxt("predictions_val_attention.txt", preds_val, fmt="%.4f")


Epoch 1 - Validation Loss: 0.5582
Epoch 2 - Validation Loss: 0.5190
Epoch 3 - Validation Loss: 0.5029
Epoch 4 - Validation Loss: 0.4843
Epoch 5 - Validation Loss: 0.4804
Epoch 6 - Validation Loss: 0.4678
Epoch 7 - Validation Loss: 0.4593
Epoch 8 - Validation Loss: 0.4636
Epoch 9 - Validation Loss: 0.4623
Epoch 10 - Validation Loss: 0.4570
Epoch 11 - Validation Loss: 0.4557
Epoch 12 - Validation Loss: 0.4525
Epoch 13 - Validation Loss: 0.4486
Epoch 14 - Validation Loss: 0.4487
Epoch 15 - Validation Loss: 0.4518
Epoch 16 - Validation Loss: 0.4737
Epoch 17 - Validation Loss: 0.4532
Epoch 18 - Validation Loss: 0.4515
Epoch 19 - Validation Loss: 0.4468
Epoch 20 - Validation Loss: 0.4455
Epoch 21 - Validation Loss: 0.4363
Epoch 22 - Validation Loss: 0.4398
Epoch 23 - Validation Loss: 0.4402
Epoch 24 - Validation Loss: 0.4449
Epoch 25 - Validation Loss: 0.4435
Epoch 26 - Validation Loss: 0.4271
Epoch 27 - Validation Loss: 0.4272
Epoch 28 - Validation Loss: 0.4346
Epoch 29 - Validation Loss: 0

100%|██████████| 14/14 [00:00<00:00, 1304.31it/s]


### MORE SEMANTIC FLAGS

In [ ]:
# --- 1. Imports ---
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
from google.colab import files

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    return np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

def assign_semantic_cluster_features(articles):
    cluster_keywords = {
        "israel_conflict": ['izraelski', 'gaza', 'palestinec', 'hamas', 'izrael', 'vojna'],
        "sports_general": ['tekma', 'liga', 'olimpija', 'prvenstvo', 'gol', 'košarka', 'nogomet', 'reprezentanca'],
        "crime": ['policija', 'kazniv', 'sodišče', 'napad', 'dejanje', 'zapor'],
        "culture": ['muzej', 'kultura', 'umetnost', 'razstava', 'galerija', 'film', 'glasba'],
        "cycling": ['dirka', 'etapa', 'kolesar', 'pogačar', 'roglič'],
        "economy": ['zakon', 'evro', 'ministrstvo', 'vlada', 'gospodarstvo', 'trg', 'energija'],
        "climate_tourism": ['voda', 'turizem', 'temperatura', 'vreme', 'okolje'],
        "nba_hockey": ['dončić', 'nba', 'dallas', 'hokej', 'rangers'],
        "politics_eu": ['stranka', 'volitev', 'parlament', 'evropski', 'predsednik'],
        "ukraine_russia": ['ruski', 'ukrajina', 'rusija', 'zelenski', 'vojna'],
        "trump_related": ['trump', 'biden', 'epstein', 'predsedniški'],
        "disasters": ['požar', 'nesreča', 'reševalec', 'gasilec', 'letalo'],
        "winter_sports": ['smučanje', 'skoki', 'pokal', 'olimpijski'],
        "energy_market": ['električen', 'energija', 'avtomobil', 'bencin', 'trg'],
        "arts_music": ['koncert', 'album', 'pesem', 'glasben', 'festival', 'nagrada'],
    }

    clusters = list(cluster_keywords.keys())
    semantic_features = np.zeros((len(articles), len(clusters)), dtype=np.float32)

    for i, article in enumerate(articles):
        text = (article.get('title', '') + ' ' + article.get('lead', '')).lower()
        for j, (cluster_name, keywords) in enumerate(cluster_keywords.items()):
            if any(kw in text for kw in keywords):
                semantic_features[i, j] = 1.0
                break
    return semantic_features

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_feats, targets):
        self.X = bert_vectors
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.semantic_feats = semantic_feats
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.semantic_feats[idx], self.y[idx]

# --- 4. Model ---
class SemanticMLPBlock(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.norm = nn.LayerNorm(input_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )

    def forward(self, x):
        x = self.norm(x)
        return self.mlp(x)

class MLPWithImprovedSemantic(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics, semantic_dim):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)
        self.semantic_mlp = SemanticMLPBlock(semantic_dim)

        total_input_dim = input_dim + 16 + 24 + 6 + 8

        self.model = nn.Sequential(
            nn.Linear(total_input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats, semantic_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        semantic_out = self.semantic_mlp(semantic_feats)

        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats, semantic_out], dim=1)
        return self.model(x)

# --- 5. Main Trainer Class ---
class RTVSloBERTImproved:
    def __init__(self, batch_size=170, epochs=150, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05, model_id=0):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_id = model_id

    def fit(self, train_data):
        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = [a['n_comments'] for a in train_data]
        targets = [np.log1p(t) for t in raw_targets]
        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in train_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(train_data)
        semantic_feats = assign_semantic_cluster_features(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, semantic_train, semantic_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )
        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        semantic_dim = semantic_feats.shape[1]
        self.model = MLPWithImprovedSemantic(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_), semantic_dim).to(self.device)

        self._train(self.model, train_loader, val_loader)

    def _train(self, model, train_loader, val_loader):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.HuberLoss(delta=5.0)

        best_val_loss = float('inf')
        patience_counter = 0
        patience_limit = 15

        for epoch in range(self.epochs):
            model.train()
            for x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

            model.eval()
            val_losses = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats, y_batch in val_loader:
                    y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze()
                    val_losses.append(criterion(y_pred, y_batch).item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            if val_loss < best_val_loss - 1e-4:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= patience_limit:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        os.makedirs("snapshots", exist_ok=True)
        torch.save(best_model_state, f"snapshots/model_{self.model_id:03d}.pt")

    def predict(self, test_data, bert_vectors):
        for a in test_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in test_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(test_data)
        semantic_feats = assign_semantic_cluster_features(test_data)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = torch.tensor(self.topic_encoder.transform(topic_ids), dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(self.subtopic_encoder.transform(subtopic_ids), dtype=torch.long).to(self.device)

        X = torch.tensor(bert_vectors, dtype=torch.float32).to(self.device)
        date_feats = torch.tensor(date_feats, dtype=torch.float32).to(self.device)
        semantic_feats = torch.tensor(semantic_feats, dtype=torch.float32).to(self.device)

        self.model.eval()
        with torch.no_grad():
            preds = self.model(X, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze().cpu().numpy()
            return np.clip(np.expm1(preds), 0, None)

# --- 6. Main Loop (Ensemble) ---
if __name__ == '__main__':
    train = load("rtvslo_train.json")
    validation = load("rtvslo_validation.json")
    test = load("rtvslo_test.json")

    bert_vectors_validation = torch.load("sloberta_embeddings_val.pt", map_location="cpu").numpy()
    bert_vectors_test = torch.load("sloberta_embeddings_final.pt", map_location="cpu").numpy()

    preds_validation = []
    preds_test = []

    for seed in range(20):
        print(f"\n🚀 Training model {seed+1}/20...")
        model = RTVSloBERTImproved(model_id=seed)
        model.fit(train)

        val_preds = model.predict(validation, bert_vectors_validation)
        test_preds = model.predict(test, bert_vectors_test)

        preds_validation.append(val_preds)
        preds_test.append(test_preds)

    final_val_preds = np.median(preds_validation, axis=0)
    final_test_preds = np.median(preds_test, axis=0)

    np.savetxt("final_predictions_val.txt", final_val_preds, fmt="%.4f")
    np.savetxt("final_predictions_test.txt", final_test_preds, fmt="%.4f")

    files.download("final_predictions_val.txt")
    files.download("final_predictions_test.txt")

    print("✅ Done!")



🚀 Training model 1/20...
Early stopping at epoch 62

🚀 Training model 2/20...
Early stopping at epoch 39

🚀 Training model 3/20...
Early stopping at epoch 60

🚀 Training model 4/20...
Early stopping at epoch 55

🚀 Training model 5/20...
Early stopping at epoch 48

🚀 Training model 6/20...
Early stopping at epoch 63

🚀 Training model 7/20...
Early stopping at epoch 76

🚀 Training model 8/20...
Early stopping at epoch 52

🚀 Training model 9/20...
Early stopping at epoch 67

🚀 Training model 10/20...
Early stopping at epoch 61

🚀 Training model 11/20...
Early stopping at epoch 63

🚀 Training model 12/20...
Early stopping at epoch 81

🚀 Training model 13/20...
Early stopping at epoch 50

🚀 Training model 14/20...
Early stopping at epoch 62

🚀 Training model 15/20...
Early stopping at epoch 62

🚀 Training model 16/20...
Early stopping at epoch 53

🚀 Training model 17/20...
Early stopping at epoch 50

🚀 Training model 18/20...
Early stopping at epoch 62

🚀 Training model 19/20...
Early stop

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done!


# AVERAGE?

In [ ]:
# --- 1. Imports ---
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
from google.colab import files

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    return np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

def assign_semantic_cluster_features(articles):
    cluster_keywords = {
        "israel_conflict": ['izraelski', 'gaza', 'palestinec', 'hamas', 'izrael', 'vojna'],
        "sports_general": ['tekma', 'liga', 'olimpija', 'prvenstvo', 'gol', 'košarka', 'nogomet', 'reprezentanca'],
        "crime": ['policija', 'kazniv', 'sodišče', 'napad', 'dejanje', 'zapor'],
        "culture": ['muzej', 'kultura', 'umetnost', 'razstava', 'galerija', 'film', 'glasba'],
        "cycling": ['dirka', 'etapa', 'kolesar', 'pogačar', 'roglič'],
        "economy": ['zakon', 'evro', 'ministrstvo', 'vlada', 'gospodarstvo', 'trg', 'energija'],
        "climate_tourism": ['voda', 'turizem', 'temperatura', 'vreme', 'okolje'],
        "nba_hockey": ['dončić', 'nba', 'dallas', 'hokej', 'rangers'],
        "politics_eu": ['stranka', 'volitev', 'parlament', 'evropski', 'predsednik'],
        "ukraine_russia": ['ruski', 'ukrajina', 'rusija', 'zelenski', 'vojna'],
        "trump_related": ['trump', 'biden', 'epstein', 'predsedniški'],
        "disasters": ['požar', 'nesreča', 'reševalec', 'gasilec', 'letalo'],
        "winter_sports": ['smučanje', 'skoki', 'pokal', 'olimpijski'],
        "energy_market": ['električen', 'energija', 'avtomobil', 'bencin', 'trg'],
        "arts_music": ['koncert', 'album', 'pesem', 'glasben', 'festival', 'nagrada'],
    }

    clusters = list(cluster_keywords.keys())
    semantic_features = np.zeros((len(articles), len(clusters)), dtype=np.float32)

    for i, article in enumerate(articles):
        text = (article.get('title', '') + ' ' + article.get('lead', '')).lower()
        for j, (cluster_name, keywords) in enumerate(cluster_keywords.items()):
            if any(kw in text for kw in keywords):
                semantic_features[i, j] = 1.0
                break
    return semantic_features

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_feats, targets):
        self.X = bert_vectors
        self.topic_ids = topic_ids
        self.subtopic_ids = subtopic_ids
        self.date_feats = date_feats
        self.semantic_feats = semantic_feats
        self.y = targets

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.semantic_feats[idx], self.y[idx]

# --- 4. Model ---
class SemanticMLPBlock(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.norm = nn.LayerNorm(input_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )

    def forward(self, x):
        x = self.norm(x)
        return self.mlp(x)

class MLPWithImprovedSemantic(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics, semantic_dim):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)
        self.semantic_mlp = SemanticMLPBlock(semantic_dim)

        total_input_dim = input_dim + 16 + 24 + 6 + 8  # BERT + topic + subtopic + date + semantic compressed

        self.model = nn.Sequential(
            nn.Linear(total_input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats, semantic_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        semantic_out = self.semantic_mlp(semantic_feats)

        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats, semantic_out], dim=1)
        return self.model(x)

# --- 5. Trainer Class ---
class RTVSloBERTImproved:
    def __init__(self, batch_size=170, epochs=150, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05, model_id=0):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_id = model_id

    def fit(self, train_data):
        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = [a['n_comments'] for a in train_data]
        targets = [np.log1p(t) for t in raw_targets]
        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in train_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(train_data)
        semantic_feats = assign_semantic_cluster_features(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, semantic_train, semantic_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, semantic_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(topic_train, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_train, dtype=torch.long).to(self.device),
            torch.tensor(date_train, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device)
        )
        val_dataset = NewsDataset(
            torch.tensor(X_val, dtype=torch.float32).to(self.device),
            torch.tensor(topic_val, dtype=torch.long).to(self.device),
            torch.tensor(subtopic_val, dtype=torch.long).to(self.device),
            torch.tensor(date_val, dtype=torch.float32).to(self.device),
            torch.tensor(semantic_val, dtype=torch.float32).to(self.device),
            torch.tensor(y_val, dtype=torch.float32).to(self.device)
        )

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        semantic_dim = semantic_feats.shape[1]
        self.model = MLPWithImprovedSemantic(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_), semantic_dim).to(self.device)

        self._train(self.model, train_loader, val_loader)

    def _train(self, model, train_loader, val_loader):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.HuberLoss(delta=5.0)

        best_val_loss = float('inf')
        patience_counter = 0
        patience_limit = 15

        for epoch in range(self.epochs):
            model.train()
            for x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

            model.eval()
            val_losses = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats, y_batch in val_loader:
                    y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze()
                    val_losses.append(criterion(y_pred, y_batch).item())

            val_loss = np.mean(val_losses)
            scheduler.step(val_loss)

            if val_loss < best_val_loss - 1e-4:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= patience_limit:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        os.makedirs("snapshots", exist_ok=True)
        torch.save(best_model_state, f"snapshots/model_{self.model_id:03d}.pt")

    def predict(self, test_data, bert_vectors):
        for a in test_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        topics, subtopics = zip(*[extract_topics_from_url(a['url']) for a in test_data])
        subtopics = [s if s != "NO_SUBTOPIC" else "none" for s in subtopics]

        date_feats = process_date_features(test_data)
        semantic_feats = assign_semantic_cluster_features(test_data)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = torch.tensor(self.topic_encoder.transform(topic_ids), dtype=torch.long).to(self.device)
        subtopic_ids = torch.tensor(self.subtopic_encoder.transform(subtopic_ids), dtype=torch.long).to(self.device)

        X = torch.tensor(bert_vectors, dtype=torch.float32).to(self.device)
        date_feats = torch.tensor(date_feats, dtype=torch.float32).to(self.device)
        semantic_feats = torch.tensor(semantic_feats, dtype=torch.float32).to(self.device)

        self.model.eval()
        with torch.no_grad():
            preds = self.model(X, topic_ids, subtopic_ids, date_feats, semantic_feats).squeeze().cpu().numpy()
            return np.clip(np.expm1(preds), 0, None)

# --- 6. Main Ensemble Loop ---
if __name__ == '__main__':
    train = load("rtvslo_train.json")
    validation = load("rtvslo_validation.json")
    test = load("rtvslo_test.json")

    bert_vectors_validation = torch.load("sloberta_embeddings_val.pt", map_location="cpu").numpy()
    bert_vectors_test = torch.load("sloberta_embeddings_final.pt", map_location="cpu").numpy()

    preds_validation = []
    preds_test = []

    for seed in range(20):
        print(f"\n🚀 Training model {seed+1}/20...")
        model = RTVSloBERTImproved(model_id=seed)
        model.fit(train)

        val_preds = model.predict(validation, bert_vectors_validation)
        test_preds = model.predict(test, bert_vectors_test)

        preds_validation.append(val_preds)
        preds_test.append(test_preds)

    final_val_preds = np.median(preds_validation, axis=0)
    final_test_preds = np.median(preds_test, axis=0)

    np.savetxt("final_predictions_val.txt", final_val_preds, fmt="%.4f")
    np.savetxt("final_predictions_test.txt", final_test_preds, fmt="%.4f")

    files.download("final_predictions_val.txt")
    files.download("final_predictions_test.txt")

    print("✅ Done!")
